# Progetto MCS

Per la gestione della struttura dati e le operazioni elementari fra matrici è richiesto di partire da una libreria open-source, come Eigen, Armadillo, blas/lapack. Oppure, qualora il linguaggio di programmazione lo permetta utilizzare vettori e matrici già implementate al suo interno.


## Data Import

In [1]:
import numpy as np
from scipy.io import mmread
from scipy.sparse import csr_matrix, csc_matrix

In [2]:
data = {
    "spa1" : 0,
    "spa2" : 0,
    "vem1" : 0,
    "vem2" : 0}
for x in data:
    data[x] = {
        "A" : mmread("data/" + x + ".mtx"),
        "x" : 0,
        "b" : 0,
    }
    data[x]["A"] = csc_matrix(data[x]["A"])
    data[x]["x"] = np.array([1.0]*data[x]["A"].get_shape()[0])
    data[x]["b"] = np.array(data[x]["A"].dot(data[x]["x"]))
tols = [10**(-4), 10**(-6), 10**(-8), 10**(-10)]

# TODO
### - Cosa che resituisca i tempi delle singole esecuzioni
### - Errore assoluto (se serve)
### - Grafici
### - Metodo preliminare per i metodi che lo hanno (es. jacobi che dice subito se la matrice è risolvibile o meno )
### - Fare file di python eseguibile
### - Sistemare e capire come raggruppare il codice

In [3]:
#jacobi
from jacoby_mcs import Jacoby
ja = Jacoby()
files_jacobi = {}
for file in data:
    jacobi_results=[]
    for tol in tols:
        jacobi_results.append(ja.execute(data[file]["A"], data[file]["b"], data[file]["x"], tol))
    files_jacobi[file] = jacobi_results

In [4]:
files_jacobi

{'spa1': [{'iter': 115, 'err_rel': 0.0017712811483049117},
  {'iter': 181, 'err_rel': 1.7979295433335858e-05},
  {'iter': 247, 'err_rel': 1.8249788763520704e-07},
  {'iter': 313, 'err_rel': 1.852437137188745e-09}],
 'spa2': [{'iter': 36, 'err_rel': 0.001766246519113358},
  {'iter': 57, 'err_rel': 1.666756136718762e-05},
  {'iter': 78, 'err_rel': 1.572869895211941e-07},
  {'iter': 99, 'err_rel': 1.4842717030144385e-09}],
 'vem1': [{'iter': 1314, 'err_rel': 0.0035403807574038567},
  {'iter': 2433, 'err_rel': 3.5400733429561766e-05},
  {'iter': 3552, 'err_rel': 3.539765956741338e-07},
  {'iter': 4671, 'err_rel': 3.539458745046825e-09}],
 'vem2': [{'iter': 1927, 'err_rel': 0.004968461406195917},
  {'iter': 3676, 'err_rel': 4.9670344320611774e-05},
  {'iter': 5425, 'err_rel': 4.965607902481058e-07},
  {'iter': 7174, 'err_rel': 4.964185220415288e-09}]}

In [5]:
#gradiente_coniugato
from gradiente_coniugato import GradC
gc = GradC()
files_gradC = {}
for file in data:
    gradC_results=[]
    for tol in tols:
        gradC_results.append(gc.execute(data[file]["A"], data[file]["b"], data[file]["x"], tol))
    files_gradC[file] = gradC_results

In [6]:
files_gradC

{'spa1': [{'iter': 49, 'err_rel': 0.020789760009958292},
  {'iter': 134, 'err_rel': 2.5529092777576548e-05},
  {'iter': 177, 'err_rel': 1.3198419641759446e-07},
  {'iter': 200, 'err_rel': 1.2136120040337842e-09}],
 'spa2': [{'iter': 42, 'err_rel': 0.009821128457265222},
  {'iter': 122, 'err_rel': 0.00011979846178594486},
  {'iter': 196, 'err_rel': 5.586660595991299e-07},
  {'iter': 240, 'err_rel': 5.324230493943813e-09}],
 'vem1': [{'iter': 38, 'err_rel': 4.082793158691563e-05},
  {'iter': 45, 'err_rel': 3.732339702308124e-07},
  {'iter': 53, 'err_rel': 2.8318734505982372e-09},
  {'iter': 59, 'err_rel': 2.1917515262943472e-11}],
 'vem2': [{'iter': 47, 'err_rel': 5.7290172222342035e-05},
  {'iter': 56, 'err_rel': 4.742996283398063e-07},
  {'iter': 66, 'err_rel': 4.299983491947605e-09},
  {'iter': 74, 'err_rel': 2.2476271758322966e-11}]}

Condizione di arresto: $\frac{||Ax^{(k)}-b||}{||b||}<tol$

Reminder: 

1.   **Iniziare le iterazioni con il vettore x nullo**
2.   **tol = [$10^{-4}, 10^{-6}, 10^{-8}, 10^{-10}]$**
3.   **Dichiarare di non essere giunti a convergenza se k > maxiter dove maxiter lo scegliamo (>= 20000)**




## Metodo di Jacobi
https://en.wikipedia.org/wiki/Jacobi_method

In [66]:
niter = 0
new_vector = np.zeros(spa1.get_shape()[0])
inverted_p_matrix = inv(np.diagflat(spa1.diagonal()))
tol = 1e-4
while True:
    residual = spa1b - spa1.dot(new_vector)
    new_vector = new_vector + (inverted_p_matrix.dot(residual))
    niter = niter +1
    if(np.linalg.norm(residual, ord=2)/np.linalg.norm(spa1b, ord=2) < tol or niter >= 20000):
      break;
print(niter)
print(new_vector)

116
[0.99997587 0.99996823 0.99996713 0.99988787 0.9999697  0.99998708
 0.99993712 0.99995962 0.9999685  0.99993124 0.99996565 0.99994862
 0.99995905 0.99992083 0.99998089 0.99997726 0.99997347 0.99991473
 0.99998243 0.99985974 0.99997079 0.99931316 0.99984283 0.99957146
 0.99989621 0.99994123 0.99996423 0.99998615 0.99968325 0.99988029
 0.99997172 0.99833472 0.9999788  0.9999122  0.99997933 0.99994805
 0.99996606 0.99997946 0.99991452 0.99998093 0.99998359 0.99989478
 0.99989794 0.99994942 0.99988924 0.9999097  0.99995751 0.99993605
 0.99970423 0.99998414 0.99996599 0.99995787 0.99998545 0.99997011
 0.99983928 0.99991803 0.99994917 0.9973644  0.99996571 0.99994407
 0.99994271 0.99990087 0.99998801 0.99997621 0.99943782 0.99998013
 0.99995413 0.99996997 0.99994514 0.99974608 0.99996764 0.99992024
 0.99997528 0.99988895 0.99996039 0.99997567 0.99983283 0.99993518
 0.99989664 0.99994993 0.99997625 0.99998138 0.99997161 0.99995571
 0.99997513 0.99996241 0.99997797 0.99992545 0.99977446 0.

In [81]:
#jacobi secondo slide del prof
niter = 0
new_vector = np.asarray([0]*len(spa1x))
inverted_p_matrix = 1/spa1.diagonal()
tol = 10**(-4)
residual = spa1b - spa1.dot(new_vector)
while np.linalg.norm(residual)/np.linalg.norm(spa1b) >= tol and niter <= 20000:
    new_vector = new_vector + (inverted_p_matrix * (residual))
    residual = spa1b - spa1.dot(new_vector)
    niter = niter +1
    print(np.linalg.norm(residual)/np.linalg.norm(spa1b))

0.2988695319547118
0.2667331051625612
0.2326557193063229
0.21925626500856593
0.19962802579240932
0.1879362969069482
0.17322691061649262
0.16256562283545192
0.150664440728867
0.1410719863068797
0.13111191603175193
0.12258790389264237
0.11410813435078872
0.10659448372372028
0.09930766441975783
0.09271753168712508
0.08642316276121635
0.0806606761542462
0.075207333681101
0.07017800747947886
0.06544508167250696
0.0610607118640954
0.056948800678557884
0.05312940397187474
0.04955482640308052
0.0462290628925069
0.043120452120045935
0.04022530763052312
0.03752131720452367
0.035001456046725006
0.032649100364795916
0.030456101819316468
0.028409483770719637
0.026501069693407427
0.02472036224487224
0.023059666815912205
0.02151027312534354
0.020065176165902553
0.018717023067482002
0.01745955314926763
0.016286489212654222
0.015192295280201088
0.014171573829989147
0.013219460616335258
0.012331293492246475
0.011502814683774391
0.010729985866797211
0.010009088717907854
0.009336619151532836
0.00870933451

### Errore relativo - numero iterazioni - tempo di calcolo

In [82]:
#numero iterazioni
niter

115

In [86]:
#errore relativo
(np.linalg.norm(np.subtract(new_vector, spa1x))/np.linalg.norm(spa1x))*100

0.17712811483049118

## Metodo di Gauß-Seidel

Preparazione dei parametri necessari al calcolo di Gauss Seidel

In [20]:
## Parameters
mtxA = data["spa1"]["A"]
maxt_iter = 20000
vectB = data["spa1"]["b"]
tol = 10**(-4)

## Creating utils variable
mtxP = mtxA.copy()
mtxN = mtxA.copy()
for i in range(len(mtxA)):
    for j in range(len(mtxA)):
        if (i<=j):
            mtxN[i][j] = 0
        else:
            mtxN[i][j] = -mtxN[i][j]
            mtxP[i][j] = 0

Implementazione del metodo iterativo

Calcolo matriciale

In [27]:
k = 0
vectX = np.random.rand(mtxA.shape[1])
#vectX = [0]*spa1.shape[0]
vectX1 = vectX.copy()
residual = np.subtract(vectB, mtxA.dot(vectX))
cond = True

while cond:
    k += 1
    vectX = vectX1.copy()
    for i in range(len(vectX1)):
        xi = vectB[i]
        for j in range(len(vectX)):
            xi -= mtxA[i][j] * vectX1[j]
        vectX1[i] = xi/mtxA[i][i]
    cond = np.linalg.norm(vectX1-vectX)/np.linalg.norm(vectX) >= tol and k <= 20000
    
    if k%100 == 0:
        print(k, end = "\t")
        if k%1000 == 0:
            print()
            print(np.linalg.norm(vectX1-vectX)/np.linalg.norm(vectX))
    

if(np.linalg.norm(residual) > tol):
    print("CIAO")
else:
    print(tol)
    print(np.linalg.norm(residual))


100	200	300	400	500	600	700	800	900	1000	
0.8764008620135874
1100	1200	1300	1400	1500	1600	1700	1800	1900	2000	
0.8964816376889414
2100	2200	2300	2400	2500	2600	2700	2800	2900	3000	
1.4588992704762889
3100	3200	3300	3400	3500	3600	3700	3800	3900	4000	
0.677207453845271
4100	4200	4300	4400	4500	4600	4700	4800	4900	5000	
0.9768569496388988
5100	5200	5300	5400	5500	5600	5700	5800	5900	6000	
0.7937177041997621
6100	6200	6300	6400	6500	6600	6700	6800	6900	7000	
0.8638600980954639
7100	7200	7300	7400	7500	7600	7700	7800	7900	8000	
0.9067438072684884
8100	8200	8300	8400	8500	8600	8700	8800	8900	9000	
0.7593186672675089
9100	9200	9300	9400	9500	9600	9700	9800	9900	10000	
1.2802618000975856
10100	10200	10300	10400	10500	10600	10700	10800	10900	11000	
0.6718528539509234
11100	11200	11300	11400	

KeyboardInterrupt: 

In [ ]:
vectX
np.linalg.norm(residual)

In [ ]:
vectX

In [ ]:
k = 0
vectX = [1] * data[x]["A"].shape[0]
residual = np.subtract(vectB, mtxA.dot(vectX))

while (np.linalg.norm(residual) > tol) and k < maxt_iter:
    k += 1
    for i in range(len(vectX)):
        xi = vectB[i]
        for j in range(len(vectX)):
            xi -= mtxA[i][j] * vectX[j]
        vectX[i] = xi/mtxA[i][i]
    residual = np.subtract(vectB, mtxA.dot(vectX))
    if k%100 == 0:
        print(k, end = "\t")
    if k%1000 == 0:
        print()
        print(np.linalg.norm(residual))

if(np.linalg.norm(residual) > tol):
    print("CIAO")
else:
    print(tol)
    print(np.linalg.norm(residual))

In [ ]:
import gauss_seidel as gs

gs.solve(mtxA=data["spa1"]["A"], vectB=data["spa1"]["b"], tol=10**(-4), maxIter=20000)

In [ ]:
len(vectX)

## Metodo del Gradiente


## Metodo del Gradiente coniugato

- Un vettore ottimale rispetto a una direzione d se d*r(k)=0
- x(k+1) è ottimale rispetto a r(k+1)
- x(k+1) = x(k) + a(k)d(k)
- a(k) = ( d(k)^t * r(k) ) / ( d(k)^t * Ad(k) )
- d(k+1) = r(k+1) - b(k)*d(k)
- b(k) = ( d(k)^t * Ar(k+1) ) / ( d(k)^t * Ad(k) )



In [103]:
#gradiente coniugato
#capire se in ak e bk bisogna calcolare la derivata di dir
niter = 0
new_vector = np.asarray([0]*len(spa1x))
tol = 10**(-4)
residual = spa1b - spa1.dot(new_vector)
dir = residual.copy()
while np.linalg.norm(residual)/np.linalg.norm(spa1b) >= tol and niter <= 20000:
    #r = spa1b - spa1.dot(new_vector)
    y = spa1.dot(dir)
    z = spa1.dot(residual)
    ak = (np.dot(dir, residual)) / (np.dot(dir, y))
    new_vector = new_vector + (ak * dir)
    residual = spa1b - spa1.dot(new_vector)
    w = spa1.dot(residual)
    bk = (np.dot(dir, w)) / (np.dot(dir, y))
    dir = residual - (bk*dir)

    residual = spa1b - spa1.dot(new_vector)
    niter = niter + 1

In [104]:
niter

49

In [102]:
np.linalg.norm(np.subtract(new_vector, spa1x))/np.linalg.norm(spa1b)

5.505564510252831e-05